### **Import Libraries**

In [ ]:
import os
import pandas as pd
import json

### **Fine-tuning**

In [ ]:
pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
training_file_name = "/content/drive/MyDrive/Research_Project/Codes/JSONGPT3.5/train.jsonl"
validation_file_name = "/content/drive/MyDrive/Research_Project/Codes/JSONGPT3.5/val.jsonl"

In [ ]:
# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id: ",training_file.id)
print("validation file id: ",validation_file.id)

In [ ]:
# Create Fine-Tuning Job
suffix_name = "GPT3.5"
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
    hyperparameters={
        "n_epochs": 5,
        "batch_size":1,
        "learning_rate_multiplier":1,
    }

)

In [ ]:
# Assuming 'response' contains the fine-tuning job details
job_id = response.id
job_details = client.fine_tuning.jobs.retrieve(job_id)
print("Job Status:", job_details.status)


In [ ]:
client.fine_tuning.jobs.list(limit=10)

In [ ]:
response = client.fine_tuning.jobs.retrieve("")#job id
response

In [ ]:
fine_tuned_model_id = response.fine_tuned_model
print('fine-tuned model id: ', fine_tuned_model_id)

### **Check Model Predictions**

In [ ]:
pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
job_id=""
response = client.fine_tuning.jobs.retrieve(job_id)#job id
response

In [ ]:
question = "A farmer has 120 acres of land. He plants corn on 3/4 of the land and soybeans on the rest. How many more acres of corn than soybeans does he plant?"

In [ ]:
def format_test(question):
    formatted_message = [{"role": "user", "content": question}]
    return formatted_message

def predict(question, fine_tuned_model_id):
    test_message = format_test(question)
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_message, temperature=1, max_tokens=250
    )
    return response.choices[0].message.content

def store_predictions(question, fine_tuned_model_id):
    prediction_result = predict(question, fine_tuned_model_id)
    return prediction_result

# question = "A multiple choice test consists of 100 questions. A correct answer is worth 2 marks, while an incorrect answer is worth -1 marks. If a student receives a score of 80, how many questions did this student answer incorrectly?"
fine_tuned_model_id = response.fine_tuned_model
model="gpt-3.5-turbo"
print('Question: ',question)
print('*****************************************************************************************')
print('')
print('Answer before fine-tuning: ')
print('')
prediction = store_predictions(question, model)
print(prediction)
print('*****************************************************************************************')
print('')
print('Answer after fine-tuning: ')
print('')
prediction = store_predictions(question, fine_tuned_model_id)
print(prediction)